# Multi-LLM Integration Demo

Test the MCP Gateway with OpenAI, Anthropic, or Google Gemini using your own API keys.

## Setup Your API Key

Set your API key as an environment variable (choose one):

In [ ]:
// Option 1: Anthropic Claude
// Deno.env.set("ANTHROPIC_API_KEY", "sk-ant-api03-...");

// Option 2: OpenAI GPT
/Deno.env.set("OPENAI_API_KEY", "sk-...");

// Option 3: Google Gemini
// Deno.env.set("GOOGLE_API_KEY", "AIza...");

console.log("⚠️  Uncomment and set your API key above before running!");

⚠️  Uncomment and set your API key above before running!


## Import LLM Provider

In [ ]:
import {
  detectProvider,
  generateCompletion,
  getDefaultModel,
} from "../examples/lib/llm-provider.ts";

const apiKey = Deno.env.get("ANTHROPIC_API_KEY") ||
  Deno.env.get("OPENAI_API_KEY") ||
  Deno.env.get("GOOGLE_API_KEY");

if (!apiKey) {
  throw new Error("No API key found! Set one in the cell above.");
}

const provider = detectProvider(apiKey);
const model = getDefaultModel(provider);

console.log(`✅ Provider: ${provider}`);
console.log(`   Model: ${model}`);

## Test 1: Simple Completion

In [ ]:
const result = await generateCompletion(
  { apiKey },
  "Explain what a code sandbox is in one sentence.",
);

console.log("Response:", result.text);
console.log("\nTokens used:", result.usage.totalTokens);

## Test 2: Code Generation with Sandbox

Use LLM to generate code, then execute it safely in sandbox:

In [ ]:
import { DenoSandboxExecutor } from "jsr:@casys/mcp-gateway";

const sandbox = new DenoSandboxExecutor({ timeout: 5000 });

// Ask LLM to generate code
const codeGenResult = await generateCompletion(
  { apiKey },
  "Write TypeScript code that calculates fibonacci(10). Return only the code, no explanation.",
  { temperature: 0.3 },
);

console.log("Generated code:");
console.log(codeGenResult.text);

// Execute the generated code safely
console.log("\nExecuting in sandbox...");
const execResult = await sandbox.execute(codeGenResult.text);

if (execResult.success) {
  console.log("✅ Result:", execResult.result);
} else {
  console.log("❌ Error:", execResult.error);
}

## Test 3: Data Analysis Assistant

LLM analyzes data, generates transformation code, sandbox executes:

In [ ]:
const salesData = [
  { product: "Laptop", revenue: 2400, units: 2 },
  { product: "Mouse", revenue: 125, units: 5 },
  { product: "Keyboard", revenue: 240, units: 3 },
];

// Ask LLM to generate analysis code
const prompt = `Given this sales data:
${JSON.stringify(salesData, null, 2)}

Write TypeScript code that:
1. Calculates total revenue
2. Finds the best-selling product by revenue
3. Returns { totalRevenue, bestProduct, avgPrice }

The data is available in context.salesData. Return only code.`;

const codeResult = await generateCompletion(
  { apiKey },
  prompt,
  { temperature: 0.2 },
);

console.log("LLM generated code:");
console.log(codeResult.text);

// Execute with actual data
const analysisResult = await sandbox.execute(
  codeResult.text,
  { salesData },
);

console.log("\n📊 Analysis Results:");
console.log(JSON.stringify(analysisResult.result, null, 2));

## Test 4: Compare Providers

If you have multiple API keys, compare responses:

In [ ]:
const providers = [
  { name: "Anthropic", key: Deno.env.get("ANTHROPIC_API_KEY") },
  { name: "OpenAI", key: Deno.env.get("OPENAI_API_KEY") },
  { name: "Google", key: Deno.env.get("GOOGLE_API_KEY") },
].filter((p) => p.key);

if (providers.length === 0) {
  console.log("⚠️  Set multiple API keys to compare providers");
} else {
  const prompt = "What is 2+2? Answer in 5 words or less.";

  console.log(`Testing ${providers.length} provider(s):\n`);

  for (const provider of providers) {
    console.log(`🤖 ${provider.name}:`);
    const result = await generateCompletion(
      { apiKey: provider.key! },
      prompt,
    );
    console.log(`   Response: ${result.text}`);
    console.log(`   Tokens: ${result.usage.totalTokens}\n`);
  }
}

## Summary

You can now:

- ✅ Use **any LLM** (OpenAI, Anthropic, Google) with your own keys
- ✅ Generate code with LLMs and execute safely in sandbox
- ✅ Build AI-powered data analysis pipelines
- ✅ Compare different providers

**Provider auto-detection:**

- `sk-ant-...` → Anthropic Claude
- `sk-...` → OpenAI GPT
- `AIza...` → Google Gemini